In [1]:
!pip install -q transformers==v4.37.1 sentencepiece datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from datasets import load_dataset, load_metric
from transformers import MT5ForConditionalGeneration, AutoTokenizer, DataCollatorForSeq2Seq

In [3]:
dataset = load_dataset("el2e10/aya-paraphrase-gujarati")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [4]:
dataset['train']

Dataset({
    features: ['inputs', 'targets', 'template_lang', 'template_id'],
    num_rows: 1001
})

In [5]:
dataset['train']['inputs'][0]

'નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "ઘટનાની જાણ થતાં પોલીસ ઘટના સ્થળે પહોંચી હતી અને ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."'

In [6]:
dataset['train']['targets'][0]

'"પોલીસ ત્યારે ઘટના સ્થળે પહોંચી હતી જ્યારે ઘટનાની જાણ થઇ હતી અને પછી ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."'

In [7]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
special_tokens = {"bos_token":"<s>"}
tokenizer.add_special_tokens(special_tokens)
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>']

# Model output test

In [9]:
input = """નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "ઘટનાની જાણ થતાં પોલીસ ઘટના સ્થળે પહોંચી હતી અને ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."""
output = """પોલીસ ત્યારે ઘટના સ્થળે પહોંચી હતી જ્યારે ઘટનાની જાણ થઇ હતી અને પછી ઘાયલોને સ્થાનિક હોસ્પિટલમાં દાખલ કરવામાં આવ્યા હતા."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")

tokenizer.batch_decode(model.generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]

'<extra_id_0> કરો'

# Dataset splits

In [10]:
datasets_train_test = dataset["train"].train_test_split(test_size=400)
datasets_train_validation = datasets_train_test["test"].shuffle().train_test_split(test_size=200)

In [11]:
datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 601
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 400
    })
})

In [12]:
datasets_train_validation

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 200
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_lang', 'template_id'],
        num_rows: 200
    })
})

In [13]:
dataset["train"] = datasets_train_test["train"]
dataset["validation"] = datasets_train_validation["train"]
test_dataset = datasets_train_validation["test"]

In [14]:
dataset["train"], dataset["validation"],test_dataset

(Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 601
 }),
 Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 200
 }),
 Dataset({
     features: ['inputs', 'targets', 'template_lang', 'template_id'],
     num_rows: 200
 }))

In [15]:
max_input_length = 512
max_target_length = 512

def preprocess_data(data):

  inputs = data["inputs"]
  targets = ["<s>" + text for text in data["targets"]]
  model_inputs = tokenizer(inputs,
                           padding=True,
                           max_length=max_input_length,
                           truncation=True)

  # Setup the tokenizer for targets
  labels = tokenizer(targets,
                     padding=True,
                     max_length=max_target_length,
                     truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [16]:
tokenized_dataset = dataset.map(preprocess_data,
                                batched=True)

Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset.keys()

dict_keys(['train', 'validation'])

In [18]:
tokenized_dataset = tokenized_dataset.remove_columns(['inputs', 'targets', 'template_lang', 'template_id'])

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

# Fine tuning

In [20]:
# parameters
BATCH_SIZE = 8
LR = 0.001
WEIGHT_DECAY = 1e-5
BETAS = (0.9,0.999)
EPSILON = 1e-8
EPOCHS = 50
EARLY_STOP_PATIENCE = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# Dataloaders
dataloader_train = torch.utils.data.DataLoader(dataset=tokenized_dataset['train'], collate_fn=data_collator, batch_size=BATCH_SIZE)
dataloader_val = torch.utils.data.DataLoader(dataset=tokenized_dataset['validation'], collate_fn=data_collator, batch_size=BATCH_SIZE)

In [22]:
# Set up tensorboard writer
writer = SummaryWriter()

# Define AdamW optimizer and ReduceLROnPlateau scheduler
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, betas=BETAS, eps=EPSILON)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Initialize early stopping variables
best_val_loss = float('inf')
early_stopping_counter = 0
model.to(DEVICE)

# Training loop
for epoch in range(EPOCHS):
    print("Train")
    total_loss = 0

    for train_batch in tqdm(dataloader_train):
        model.train()
        train_batch_inputs = {k: v.to(DEVICE) for k, v in train_batch.items()}

        optimizer.zero_grad()
        logits = model(**train_batch_inputs).logits
        loss = criterion(logits.view(-1, logits.shape[-1]), train_batch_inputs["labels"].view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        writer.add_scalar('Train Loss', loss.item(), epoch)

    average_loss = total_loss / len(dataloader_train)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Train Loss: {average_loss}')

    print("Val")
    total_val_loss = 0

    for val_batch in tqdm(dataloader_val):
        model.eval()
        val_batch_inputs = {k: v.to(DEVICE) for k, v in val_batch.items()}

        val_logits = model(**val_batch_inputs).logits
        val_loss = criterion(val_logits.view(-1, val_logits.shape[-1]), val_batch_inputs["labels"].view(-1))

        total_val_loss += val_loss.item()
        writer.add_scalar('Validation Loss', loss.item(), epoch)

    average_val_loss = total_val_loss / len(dataloader_val)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Validation Loss: {average_val_loss}')

    # Update learning rate using the scheduler
    lr_scheduler.step(average_val_loss)

    # Check for early stopping
    if average_val_loss < best_val_loss:
        best_val_loss = average_val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= EARLY_STOP_PATIENCE:
            print(f'Early stopping after {epoch + 1} epochs without improvement.')
            model.save_pretrained(f'/content/drive/MyDrive/gujju-gpt/Models/mT5-gujarati-summarization_ep{epoch+1}')
            break
# Close tensorboard writer
writer.close()

Train


  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 1/50, Train Loss: 5.344256325771934
Val


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/50, Validation Loss: 1.0142884254455566
Train


...

Epoch 28/50, Validation Loss: 0.5057976126670838
Train


  0%|          | 0/76 [00:00<?, ?it/s]

Epoch 29/50, Train Loss: 0.24012622340141157
Val


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 29/50, Validation Loss: 0.514696307182312
Early stopping after 29 epochs without improvement.


In [23]:
test_dataset[7]

{'inputs': 'નીચેના વાક્યને અલગ શબ્દોનો ઉપયોગ કરીને લખો: "ભારત અને પાકિસ્તાન સાતમી વખત વર્લ્ડ કપમાં આમને-સામને છે."',
 'targets': '"પાકિસ્તાન અને ભારત વર્લ્ડ કપમાં સાતમી વાર એકબીજા સામે છે."',
 'template_lang': "['guj']",
 'template_id': 1}

In [24]:
reference_summary = """નીચેના વાક્યને અલગ રીતે ફરીથી લખો: "કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કરી છે."""
input = """કેરળમાં આવેલા પૂરને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને કરી છે."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")
hypothesis_summary = tokenizer.batch_decode(model.to('cpu').generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]
hypothesis_summary

'"કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ પીએમ મોદીને રાષ્ટ્રીય આપત્તિ જાહેર કરવાની માંગ કોંગ્રેસ અધ્યક્ષ રાહુલ ગાંધીએ કરી છે."'

In [25]:
test_dataset[10]

{'inputs': 'નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "આ વિસ્ફોટમાં નવપરિણીત સૌમ્યા શેખર સાહુ અને તેની દાદીનું મૃત્યુ થયું હતું જ્યારે તેની દુલ્હન રીમાને ગંભીર ઈજા પહોંચી હતી."',
 'targets': '"નવપરિણીત સૌમ્યા શેખર સાહુ અને તેની દાદીનું આ વિસ્ફોટમાં મૃત્યુ થયું હતું જ્યારે તેની દુલ્હન રીમાને ગંભીર ઈજા પહોંચી હતી."',
 'template_lang': "['guj']",
 'template_id': 3}

In [26]:
reference_summary = """નીચેના વાક્યને બીજા સ્વરૂપમાં ફરીથી લખો: "22 વર્ષની ઉંમરે, પંતે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."""
input = """પંતે 22 વર્ષની ઉંમરે પહેલાંથી જ ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 15 ઓડિસ અને 28 ટી 20 આઈ રમ્યા છે."""
inputs = tokenizer(input, text_target=output, return_tensors="pt")
hypothesis_summary = tokenizer.batch_decode(model.to('cpu').generate(inputs.input_ids,max_length=50),skip_special_tokens=True)[0]
hypothesis_summary

'"ભારતીય ક્રિકેટ ટીમ માટે 13 ટેસ્ટ, 14 ઓડિસ અને 28 ટી-20 આઈ રમ્યા છે."'

# Test results

In [27]:
model = MT5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/gujju-gpt/Models/mT5-gujarati-summarization_ep29")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [28]:
test_dataset

Dataset({
    features: ['inputs', 'targets', 'template_lang', 'template_id'],
    num_rows: 200
})

In [29]:
ref_summary = []
gen_summary = []

In [30]:
output_file = "mT5-gujarati-summarization-test-results-ep29.txt"
with open(output_file, 'w', encoding='utf-8') as file:
    for i, item in enumerate(test_dataset):
        inputs = item['inputs']
        targets = item['targets']
        model_input = tokenizer(inputs, return_tensors="pt").to('cuda')
        hypothesis_summary = tokenizer.batch_decode(
            model.to('cuda').generate(model_input.input_ids,max_length=50),
            skip_special_tokens=True)[0]

        ref_summary.append(targets)
        gen_summary.append(hypothesis_summary)



        # Write to the file in the desired format
        file.write(f'Sample No. : {i}\n')
        file.write(f'Inputs: {inputs}\n')
        file.write(f'Targets: {targets}\n')
        file.write(f'Generated: {hypothesis_summary}\n\n')

In [31]:
# notebook workaround if you get error : "A UTF-8 locale is required. Got ANSI_X3.4-1968" while using !pip install
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [32]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d7d21f26ad872e0cf6b8701084a2d6925ee8c75296d340936a89d73d7044e8e3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [33]:
from rouge_score import rouge_scorer

# Create a RougeScorer object
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize variables to accumulate scores
total_scores = {"rouge1": {"precision": 0, "recall": 0, "fmeasure": 0},
                "rouge2": {"precision": 0, "recall": 0, "fmeasure": 0},
                "rougeL": {"precision": 0, "recall": 0, "fmeasure": 0}}

# Iterate over samples and accumulate scores
for reference_summary, hypothesis_summary in zip(ref_summary, gen_summary):
    scores = scorer.score(reference_summary, hypothesis_summary)
    for metric in total_scores:
        total_scores[metric]["precision"] += scores[metric].precision
        total_scores[metric]["recall"] += scores[metric].recall
        total_scores[metric]["fmeasure"] += scores[metric].fmeasure

# Calculate average scores
num_samples = len(ref_summary)
average_scores = {metric: {measure: total_scores[metric][measure] / num_samples for measure in ["precision", "recall", "fmeasure"]} for metric in total_scores}

# Print the aggregated ROUGE scores
print("Aggregated ROUGE Scores:")
print(average_scores)


Aggregated ROUGE Scores:
{'rouge1': {'precision': 0.05208333333333333, 'recall': 0.04716666666666666, 'fmeasure': 0.048999999999999995}, 'rouge2': {'precision': 0.024166666666666666, 'recall': 0.02125, 'fmeasure': 0.022261904761904764}, 'rougeL': {'precision': 0.05208333333333333, 'recall': 0.04716666666666666, 'fmeasure': 0.048999999999999995}}
